In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(os.environ["CUDA_VISIBLE_DEVICES"],torch.cuda.device_count(), torch.cuda.current_device())

3 1 0


In [2]:
import os
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
# from utils import *
# from modules import UNet
import logging
from torch.utils.tensorboard import SummaryWriter

import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
from torch import optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import math

print(torch.__version__)
torch.cuda.is_available()

import h5py


1.12.0+cu102


In [19]:
class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=[128, 2, 2], device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    # def sample(self, model, n):
    #     logging.info(f"Sampling {n} new images....")
    #     model.eval()
    #     with torch.no_grad():
    #         x = torch.randn((n, self.img_size[0], self.img_size[1],self.img_size[2])).to(self.device)
    #         for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
    #             t = (torch.ones(n) * i).long().to(self.device)
    #             predicted_noise = model(x, t)
    #             alpha = self.alpha[t][:, None, None, None]
    #             alpha_hat = self.alpha_hat[t][:, None, None, None]
    #             beta = self.beta[t][:, None, None, None]
    #             if i > 1:
    #                 noise = torch.randn_like(x)
    #             else:
    #                 noise = torch.zeros_like(x)
    #             x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
    #     model.train()
    #     # x = (x.clamp(-1, 1) + 1) / 2
    #     x = x.clamp(0, 1)
    #     # x = (x * 255).type(torch.uint8)
    #     return x
    
    def sample(self, model, n, labels, cfg_scale=3):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, self.img_size[0], self.img_size[1],self.img_size[2])).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                predicted_noise = model(x, t, labels)
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        model.train()
        # x = (x.clamp(-1, 1) + 1) / 2
        # x = (x * 255).type(torch.uint8)
        # x = x.clamp(0, 1)
        return x

In [6]:
is_wandb = True
if is_wandb:
    import wandb
    wandb.init(project="diff_protein_gen", entity="jathurshan_0330")
    wandb.run.name = "Test_with_condition_2"
    wandb.run.save()
    # "Finetuning only classification head Ki67 using SIMCLR Pretrained Model"
    exp_path = f"./model_checkpoints/{wandb.run.name}"
    if not os.path.exists(exp_path):
        os.mkdir(exp_path)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jathurshan_0330. Use `wandb login --relogin` to force relogin
/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.




wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


## Data Loading

In [7]:
from torchvision.transforms import ToTensor 
mnist_train_dataset = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = transforms.Compose([ToTensor(), transforms.Resize(32)]),
    download = True,            
)
mnist_test_dataset = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = transforms.Compose([ToTensor(), transforms.Resize(32)]),
)

In [8]:
train_data_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=32, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=32, shuffle=False)

## Autoencoder Model

In [9]:
class CNN_Encoder(nn.Module):
    
    def __init__(self, input_nc=1, ngf = 16):
        super(CNN_Encoder, self).__init__()
        self.enc1 = self.enc_block(in_ch = input_nc, out_ch = ngf, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        self.enc2 = self.enc_block(in_ch = ngf, out_ch = ngf*2, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        # self.enc3 = self.enc_block(in_ch = ngf*2, out_ch = ngf*4, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        # self.enc4 = self.enc_block(in_ch = ngf*4, out_ch = ngf*8, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        # self.enc5 = self.enc_block(in_ch = ngf*8, out_ch = ngf*16, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        
        
    def enc_block(self, in_ch, out_ch, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size,stride=stride, padding=padding, bias=bias),
                # nn.ReLU()
                )
      
            
    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        # x = self.enc3(x)
        # x = self.enc4(x)
        # x = self.enc5(x)
        return x
    

class CNN_Decoder(nn.Module):
    
    def __init__(self, lat_chan=128,out_ch = 1, ngf = 16 ):
        super(CNN_Decoder, self).__init__()
        self.dec1 = self.conv_up_block(in_ch = lat_chan , out_ch = ngf*8, 
                                       kernel_size=4, stride=2, padding = 1,bias = True)
        self.dec2 = self.conv_up_block(in_ch = ngf*8 , out_ch = 1, 
                                       kernel_size=4, stride=2, padding = 1,bias = True)
        # self.dec3 = self.conv_up_block(in_ch = ngf*4 , out_ch = ngf*2, 
        #                                kernel_size=4, stride=2, padding = 1,bias = True)
        # self.dec4 = self.conv_up_block(in_ch = ngf*2 , out_ch = 1, 
        #                                kernel_size=4, stride=2, padding = 1,bias = True)
        # self.dec5 = self.conv_up_block(in_ch = ngf , out_ch = 1, 
                                       # kernel_size=4, stride=2, padding = 1,bias = True)
        # self.final = nn.Linear(16,out_ch)
        
        
    def conv_up_block(self,  in_ch, out_ch, kernel_size=4, stride=2, padding = 1,bias = True,outermost = False):
            return nn.Sequential(
                nn.ConvTranspose2d( in_ch, out_ch, kernel_size=kernel_size, stride=stride, padding = padding,bias = bias),
                # nn.ReLU()
                )
        
    def forward(self,x): 
        x = self.dec1(x)
        
        x = self.dec2(x)
        
        
#         x = self.dec3(x)
        
#         x = self.dec4(x)
        # x = self.dec5(x)
        # x = self.final(x)
        # x = torch.moveaxis(x,-1,1)
        return x

In [10]:
enc_path = f'./model_checkpoints/autoencoder_MNIST/encoder_2.pth.tar'
encoder = torch.load(enc_path).to(device)#CNN_Encoder(ngf = 16).to(device)#torch.load(enc_path).to(device)
encoder.eval()
print(sum([np.prod(p.size()) for p in encoder.parameters()]))

dec_path = f'./model_checkpoints/autoencoder_MNIST/decoder_2.pth.tar'
decoder = torch.load(dec_path).to(device)#CNN_Decoder(lat_chan=32).to(device)#torch.load(dec_path).to(device)
decoder.eval()
print(sum([np.prod(p.size()) for p in decoder.parameters()]))

8496
67713


## UNet

In [11]:
class EMA:
    def __init__(self, beta):
        super().__init__()
        self.beta = beta
        self.step = 0

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

    def step_ema(self, ema_model, model, step_start_ema=2000):
        if self.step < step_start_ema:
            self.reset_parameters(ema_model, model)
            self.step += 1
            return
        self.update_model_average(ema_model, model)
        self.step += 1

    def reset_parameters(self, ema_model, model):
        ema_model.load_state_dict(model.state_dict())


class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.ff_self = nn.Sequential(
            nn.LayerNorm([channels]),
            nn.Linear(channels, channels),
            nn.GELU(),
            nn.Linear(channels, channels),
        )

    def forward(self, x):
        x = x.view(-1, self.channels, self.size * self.size).swapaxes(1, 2)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        attention_value = self.ff_self(attention_value) + attention_value
        return attention_value.swapaxes(2, 1).view(-1, self.channels, self.size, self.size)


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        self.residual = residual
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, mid_channels),
            nn.GELU(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(1, out_channels),
        )

    def forward(self, x):
        if self.residual:
            return F.gelu(x + self.double_conv(x))
        else:
            return self.double_conv(x)


class Down(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, t):
        x = self.maxpool_conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb


class Up(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.conv = nn.Sequential(
            DoubleConv(in_channels, in_channels, residual=True),
            DoubleConv(in_channels, out_channels, in_channels // 2),
        )

        self.emb_layer = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_dim,
                out_channels
            ),
        )

    def forward(self, x, skip_x, t):
        x = self.up(x)
        x = torch.cat([skip_x, x], dim=1)
        x = self.conv(x)
        emb = self.emb_layer(t)[:, :, None, None].repeat(1, 1, x.shape[-2], x.shape[-1])
        return x + emb


class UNet(nn.Module):
    def __init__(self, c_in=3, c_out=3, time_dim=256, device="cuda"):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, 4)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, 2)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, 1)

        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 512)
        self.bot3 = DoubleConv(512, 256)

        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, 2)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, 4)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64, 8)
        self.outc = nn.Conv2d(64, c_out, kernel_size=1)

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2, device=self.device).float() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)

        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        # print(f"x2: {x2.shape}")
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        # print(f"x3: {x3.shape}")
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        # print(f"x4: {x4.shape}")
        x4 = self.sa3(x4)
        
        # print(f"x4: {x4.shape}")
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        x4 = self.bot3(x4)
        
        # print(f"x4: {x4.shape}")
        x = self.up1(x4, x3, t)
        # print(f"x: {x.shape}")
        x = self.sa4(x)
        # print(f"x: {x.shape}")
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.outc(x)
        return output
    
    
class UNet_Conditional(nn.Module):
    def __init__(self, c_in=3, c_out=3, time_dim=256, device="cuda",num_classes = None):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, 4)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, 2)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, 1)

        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 512)
        self.bot3 = DoubleConv(512, 256)

        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, 2)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, 4)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64, 8)
        self.outc = nn.Conv2d(64, c_out, kernel_size=1)
        
        if num_classes is not None:
            self.label_emb = nn.Embedding(num_classes, time_dim)
        

    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (
            10000
            ** (torch.arange(0, channels, 2, device=self.device).float() / channels)
        )
        pos_enc_a = torch.sin(t.repeat(1, channels // 2) * inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels // 2) * inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc

    def forward(self, x, t,labels):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)
        # print(f"t:{t.shape}")
        if labels!=None:
            # print(self.label_emb(labels).shape)
            t = t+ self.label_emb(labels)
        
        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        # print(f"x2: {x2.shape}")
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        # print(f"x3: {x3.shape}")
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        # print(f"x4: {x4.shape}")
        x4 = self.sa3(x4)
        
        # print(f"x4: {x4.shape}")
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        x4 = self.bot3(x4)
        
        # print(f"x4: {x4.shape}")
        x = self.up1(x4, x3, t)
        # print(f"x: {x.shape}")
        x = self.sa4(x)
        # print(f"x: {x.shape}")
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        output = self.outc(x)
        return output

In [12]:
# class Cond_Encoder(nn.Module):
    
#     def __init__(self, input_nc=1, ngf = 64):
#         super(Cond_Encoder, self).__init__()
#         self.enc1 = self.enc_block(in_ch = input_nc, out_ch = ngf, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
#         self.enc2 = self.enc_block(in_ch = ngf, out_ch = ngf*2, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
#         self.enc3 = self.enc_block(in_ch = ngf*2, out_ch = ngf*4, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
#         # self.enc4 = self.enc_block(in_ch = ngf*4, out_ch = ngf*8, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
#         # self.enc5 = self.enc_block(in_ch = ngf*8, out_ch = ngf*16, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False  )
        
        
#     def enc_block(self, in_ch, out_ch, kernel_size=4, stride=2, padding = 1,bias = True, innermost = False):
#             return nn.Sequential(
#                 nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size,stride=stride, padding=padding, bias=bias),
#                 nn.BatchNorm1d(out_ch),
#                 # nn.ReLU(),
#                 )
      
            
#     def forward(self, x):
#         x = self.enc1(x)
#         x = self.enc2(x)
#         x = self.enc3(x)
#         # x = self.enc4(x)
#         # x = self.enc5(x)
#         return torch.sigmoid(x)

In [13]:
digit_imgs,digit_labels   = next(iter(train_data_loader))
# print(digit_labels)
digit_labels = F.one_hot(digit_labels, num_classes=10)
print(digit_labels.shape)

torch.Size([32, 10])


In [14]:
model = UNet_Conditional(c_in=32, c_out=32, time_dim=256, device="cuda",num_classes = 10).to(device)
# cond_enc = Cond_Encoder().to(device)
#testing
digit_imgs,digit_labels   = next(iter(train_data_loader))
# digit_labels = F.one_hot(digit_labels, num_classes=10).unsqueeze(1).float()
# cond_vec = cond_enc(digit_labels.to(device))
print(digit_labels.shape)

lat_c = encoder(digit_imgs.to(device))
t = torch.randint(low=1, high=1000, size=(digit_imgs.shape[0],)).to(device)
print(lat_c.shape,t.shape)

denoised_lat_c = model(lat_c,t,digit_labels.to(device))

print(denoised_lat_c.shape)

torch.Size([32])
torch.Size([32, 32, 8, 8]) torch.Size([32])
torch.Size([32, 32, 8, 8])


## Training

In [21]:
# optimizer = optim.AdamW(list(model.parameters())+list(cond_enc.parameters()), lr=3e-4)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
mse = nn.MSELoss()
diffusion = Diffusion(img_size=[32,8,8], device=device)

In [22]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count if self.count != 0 else 0
        
def plot_sub_plots(rows,columns,img_list,title_list,cmap = 'inferno'):
    fig, axs = plt.subplots(rows, columns, figsize = (columns*10,rows*10))
    if rows == 1:
        for i in range(len(img_list)):
            im1 = axs[i].imshow(img_list[i],cmap=cmap)
            axs[i].set_title(f"{title_list[i]}")
            plt.colorbar(im1, ax=axs[i])#,shrink = 0.3
    else:
        for i in range(len(img_list)):
            im1 = axs[i//columns][i%columns].imshow(img_list[i],cmap=cmap)
            axs[i//columns][i%columns].set_title(f"{title_list[i]}")
            plt.colorbar(im1, ax=axs[i//columns][i%columns])#,shrink = 0.3)
    # plt.show()
    return fig

In [23]:
def train(model,diffusion,encoder,decoder,optimizer,data_loader,is_wandb=False,verbose_freq = 500,is_verbose = False):
    encoder.eval()
    decoder.eval()
    model.train()
    # cond_enc.train()
    train_losses = AverageMeter()
    for batch_idx, (digit_imgs,digit_labels ) in enumerate(data_loader):
        images = encoder(digit_imgs.to(device))
        
        # digit_labels = F.one_hot(digit_labels, num_classes=10).unsqueeze(1).float()
        # cond_vec = cond_enc(digit_labels.to(device))
        
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        predicted_noise = model(x_t, t,digit_labels.to(device))
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      
        train_losses.update(loss.data.item())
        if is_wandb:
            wandb.log({"batch_loss": loss.data.item()})
    
    
    if is_wandb:
            wandb.log({"train_epoch_loss": train_losses.avg})
            
    if is_verbose:
        print(f"Training Epoch Loss: {train_losses.avg}")
    
    digit_imgs,digit_labels   = next(iter(data_loader))
    # cond_enc.eval()
    # digit_labels_2 = F.one_hot(digit_labels, num_classes=10).unsqueeze(1).float()
    # cond_vec = cond_enc(digit_labels_2.to(device))
    
    sampled_images = diffusion.sample(model, n=images.shape[0],labels = digit_labels.to(device))
    sampled_images = decoder(sampled_images.float())
    sampled_images = sampled_images.squeeze().detach().cpu().numpy()
    print(sampled_images.shape)
    
    fig = plot_sub_plots(4,8,sampled_images,[str(digit_labels[i].item()) for i in range(32)],cmap = 'inferno')
    if is_wandb:
            wandb.log({"examples": wandb.Image(fig)})
    
    return train_losses.avg





    # sampled_images = diffusion.sample(model, n=images.shape[0])
    # save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch}.jpg"))
    # torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))


In [ ]:
best_loss = 10000000
train_loss_list = []
n_epochs = 100

# if not os.path.exists(exp_path):
#     os.mkdir(exp_path)
# print("Training ", end='')
for epoch_idx in range(n_epochs):
    # print('=', end='')
    
    train_loss = train(model,diffusion,encoder,decoder,optimizer,train_data_loader,is_wandb=is_wandb)     

    train_loss_list.append(train_loss)
    
    
    print(f"Epoch {epoch_idx+1}: {train_loss} ")
    if (train_loss < best_loss): 
        best_loss = train_loss
        # print("Saving Best Model =======================================>")
        torch.save(model, f'{exp_path}/u_net_diff.pth.tar')
        # torch.save(cond_enc, f'{exp_path}/cond_enc_diff.pth.tar')
    
                       
                       

999it [00:17, 57.60it/s]


(32, 32, 32)
Epoch 1: 0.0129135233014822 


999it [00:17, 57.00it/s]


(32, 32, 32)
Epoch 2: 0.012545379406958818 


999it [00:17, 56.89it/s]


...

999it [00:17, 56.13it/s]


(32, 32, 32)
Epoch 83: 0.010183234094580015 


999it [00:17, 55.89it/s]


(32, 32, 32)
Epoch 84: 0.010205492693434159 


106it [00:01, 60.29it/s]